# Overview of the Notebook
This notebook is designed to test and evaluate a recommender model for customer support ticketing. The model retrieves relevant complaints and generates responses based on a given query. Additionally, we compare the performance of this model with the DeepSeek-R1 model (1.5B parameters).

In order to use this notebook and the models, you need to download Ollama from [https://ollama.com/download](https://ollama.com/download) and pull the following models: 
1. llama3.2
1. mxbai-embed-large
1. deepseek-r1:1.5b.

## Importing Required Libraries
This cell imports all the necessary libraries and modules required for the notebook, including libraries for language models, vector storage, and data processing.

In [9]:
# Import necessary libraries
import os
import sys
import pandas as pd
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings

## Model Initialization
This cell initializes the language model (Llama) for generating responses, the embedding model for vector retrieval, and the vector store for storing and retrieving complaint data.

In [10]:
# Initialize models and configurations
# LLM MODEL FOR GENERATING RESPONSES

model = OllamaLLM(model="llama3.2")

# EMBEDDING MODEL FOR RETRIEVAL

embeddings = OllamaEmbeddings(model="mxbai-embed-large")

# Chroma VECTOR STORE FOR RETRIEVAL

db_location = "./chrome_langchain_db"

# Load the CSV file into a DataFrame used for retrieval
df = pd.read_csv(
    "archive/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11 (1).csv")


    
vector_store = Chroma(
    collection_name="problems_and_solutions",
    persist_directory=db_location,
    embedding_function=embeddings
)




## Loading and Adding Data to the Vector Store
This cell loads the customer support dataset from a CSV file and adds the data to the vector store if it is not already populated.

In [11]:
add_documents = not os.path.exists(
    db_location)  # if the directory is empty

if add_documents:
    documents = []
    ids = []
    for i, row in df.iterrows():
        if i == 5000:
            break
        document = Document(
            page_content=row["instruction"] + " " + row["response"],
            metadata={"category": row["category"], "intent": row["intent"]},
            id=str(i)
        )
        ids.append(str(i))
        documents.append(document)

    print(f"Adding {len(documents)} documents to the vector store")
    vector_store.add_documents(documents=documents, ids=ids)

## Creating a Retriever
This cell creates a retriever from the vector store, which retrieves the top 5 most relevant documents based on a query.

In [12]:
# Create a retriever from the vector store that retrieves the top 5 documents based on similarity to the query
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

## Defining the Chat Function
This cell defines the `chat_with_model` function, which takes a user query, retrieves relevant complaints, and generates a response using the Llama model.

In [13]:
# Define the chat function
template = """
You are a customer ticket support AI Assistant and your job is only to instruct users on how to solve their problems.
this is not a chat but only a reply.
ensure the customer that there will be an employee replying to him as well soon.

Here are some relevant problems and how to handle them: {problems}

Here is the question to answer: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model

def chat_with_model(question: str):
    problems = retriever.invoke(question)
    for idx,problem in enumerate(problems):
        print(f"Relevant Complaint {idx}: {problem.page_content}\n\n")

    result = chain.invoke({"problems": problems, "question": question})
    return result

## Defining Test Cases
This cell defines a list of test cases (queries) to evaluate the performance of the recommender model.

In [14]:
# Define test cases
test_cases = [
    # "How can I reset my password?",
    "I want to cancel my order no 123456"
    # "What should I do if my product is defective?",
    # "How do I update my account information?"
]

## Running the Model
This cell runs the `chat_with_model` function for each test case and prints the generated responses.

In [15]:
for test_case in test_cases:
    print("------------------------------------------\n")
    print(f"Test case:\n {test_case}\n")
    response = chat_with_model(test_case)
    print(f"Generated Response:\n {response}\n")

------------------------------------------

Test case:
 I want to cancel my order no 123456

Relevant Complaint 0: i purchased some fucking item cancel order {{Order Number}} I'm aware that you're frustrated with your purchase and would like to cancel order {{Order Number}}. I apologize for any inconvenience caused and assure you that I'm here to assist you through this process. To cancel your order, please follow these steps:

1. Access Your Account: Log in to your {{Online Company Portal Info}} using your credentials.
2. Navigate to Your Orders: Once you're logged in, locate the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section.
3. Find Your Specific Purchase: Look for the order with the number {{Order Number}} and click on it to view the details.
4. Initiate the Cancellation: Within the order details, you should find an option to '{{Online Order Interaction}}' or similar. Please select that to start the cancellation process.
5. Confirm the Cancellation: The sy

## Comparison with DeepSeek-R1 Model
This cell introduces the comparison between the Llama model and the DeepSeek-R1 model (1.5B parameters).

In [8]:

model =OllamaLLM(model="deepseek-r1:1.5b")
chain = prompt | model

def chat_with_model(question: str):
    problems = retriever.invoke(question)
    for idx,problem in enumerate(problems):
        print(f"Relevant Complaint {idx}: {problem.page_content}\n\n")

    result = chain.invoke({"problems": problems, "question": question})
    return result

for test_case in test_cases:
    print("------------------------------------------\n")
    print(f"Test case:\n {test_case}\n")
    response = chat_with_model(test_case)
    print(f"Generated Response:\n {response.split('</think>')[-1]}\n")

------------------------------------------

Test case:
 I want to cancel my order no 123456

Relevant Complaint 0: i purchased some fucking item cancel order {{Order Number}} I'm aware that you're frustrated with your purchase and would like to cancel order {{Order Number}}. I apologize for any inconvenience caused and assure you that I'm here to assist you through this process. To cancel your order, please follow these steps:

1. Access Your Account: Log in to your {{Online Company Portal Info}} using your credentials.
2. Navigate to Your Orders: Once you're logged in, locate the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section.
3. Find Your Specific Purchase: Look for the order with the number {{Order Number}} and click on it to view the details.
4. Initiate the Cancellation: Within the order details, you should find an option to '{{Online Order Interaction}}' or similar. Please select that to start the cancellation process.
5. Confirm the Cancellation: The sy

## Why We Prefer the Llama Model's Answer

The Llama model's response is preferred because it adheres closely to the instructions provided in the prompt. Specifically, the Llama model ensures that:

1. **Acknowledgment of Employee Assistance**: The Llama model explicitly mentions that another employee will respond to the ticket soon, as instructed in the prompt. This reassurance is important for maintaining customer trust and setting expectations.

2. **Prompt Adherence**: The Llama model strictly follows the prompt's directive to provide a reply that is not conversational but instructional, ensuring clarity and professionalism in its response.

In contrast, the DeepSeek-R1 model ignored the instruction to mention the involvement of another employee. This omission makes its response less aligned with the desired behavior outlined in the prompt, potentially leading to a less satisfactory customer experience.

By following the prompt more accurately, the Llama model demonstrates better alignment with the intended use case and provides a more complete and reassuring response to the customer.